# Imports and MRI Data Loading

In [ ]:
!pip install nilearn
!pip install -q kaggle

#Imports for preprocessing, scikitlearn model imports still needed
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
import random
from nilearn.masking import compute_epi_mask
from nilearn.masking import apply_mask
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import zipfile
from google.colab import files


In [ ]:
from google.colab import files

import os, zipfile

# 1) Upload kaggle.json
print("Upload your kaggle.json file")
uploaded = files.upload()

# 2) Find the uploaded filename dynamically
kaggle_filename = list(uploaded.keys())[0]
print("Using uploaded file:", kaggle_filename)

# 3) Put it where Kaggle CLI expects it
kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

with open(os.path.join(kaggle_dir, "kaggle.json"), "wb") as f:
    f.write(uploaded[kaggle_filename])

# 4) Fix permissions
!chmod 600 ~/.kaggle/kaggle.json

DATA_DIR = "/content/brats2020"
os.makedirs(DATA_DIR, exist_ok=True)

zip_path = os.path.join(DATA_DIR, "brats20-dataset-training-validation.zip")
extracted_dir = os.path.join(DATA_DIR, "BraTS2020_TrainingData")

# Download only if zip is missing
if not os.path.exists(zip_path):
    print("Downloading dataset from Kaggle...")
    !kaggle datasets download -d awsaf49/brats20-dataset-training-validation -p $DATA_DIR
else:
    print("Zip already exists, skipping download.")

!ls -lh $DATA_DIR

# Only unzip if we don't see the extracted folder yet
if not os.path.isdir(extracted_dir):
    print("Extracting zip... this can take a few minutes.")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)
    print("Extraction complete.")
    # Optional: delete zip to save space
    os.remove(zip_path)
    print("Removed zip file.")
else:
    print("Found extracted directory, skipping unzip.")


In [ ]:
!pip install nilearn
!pip install -q numpy pandas matplotlib nibabel nilearn scikit-learn scikit-fuzzy scipy
!pip install -q seaborn tqdm

In [ ]:
TEST_MODE = True  # Set to False for full dataset
MAX_TEST_PATIENTS = 45

if TEST_MODE:
    print(f"TEST MODE: Will process only {MAX_TEST_PATIENTS} patients")
else:
    print("FULL MODE: Processing all patients")

In [ ]:
# Imports for preprocessing, scikitlearn model imports still needed
import os, random, warnings, zipfile
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment

import seaborn as sns
from tqdm import tqdm
from nilearn.masking import apply_mask
from IPython.display import display

In [ ]:
# Load and Organize files
DATA_ROOT = "/content/brats2020"

TRAIN_DIR = os.path.join(
    DATA_ROOT, "BraTS2020_TrainingData", "MICCAI_BraTS2020_TrainingData"
)

print("Train dir:", TRAIN_DIR)

def load_patient(patient_dir):
    """Load all modalities for a single BraTS patient."""
    pid = os.path.basename(patient_dir)

    def load_mod(mod):
        # Kaggle BraTS files are .nii, not .nii.gz
        f = os.path.join(patient_dir, f"{pid}_{mod}.nii")
        return nib.load(f).get_fdata().astype(np.float32)

    flair = load_mod("flair")
    t1    = load_mod("t1")
    t1ce  = load_mod("t1ce")
    t2    = load_mod("t2")
    seg   = load_mod("seg")

    return {
        "PID": pid,
        "FLAIR": flair,
        "T1": t1,
        "T1CE": t1ce,
        "T2": t2,
        "SEG": seg,
    }

patients = {}
masked_patients = {}

patient_list = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])

if TEST_MODE:
    patient_list = patient_list[:MAX_TEST_PATIENTS]
    print(f"TEST MODE: Processing only first {MAX_TEST_PATIENTS} patients\n")

for patient_id in patient_list:
    patient_path = os.path.join(TRAIN_DIR, patient_id)

    try:
        patient_data = load_patient(patient_path)
        patients[patient_id] = {
            "T1": patient_data["T1"],
            "T1CE": patient_data["T1CE"],
            "T2": patient_data["T2"],
            "FLAIR": patient_data["FLAIR"],
            "SEG": patient_data["SEG"]
        }
        print(f"Loaded {patient_id} → {list(patients[patient_id].keys())}")
    except Exception as e:
        print(f"Skipped {patient_id} (error: {e})")

print(f"\n Total patients loaded: {len(patients)}")
if len(patients) == 0:
    print(f"WARNING: No patients loaded! Check the directory structure.")
    print(f"Contents of {TRAIN_DIR}:")
    if os.path.exists(TRAIN_DIR):
        print(os.listdir(TRAIN_DIR)[:10])  # Show first 10 items
    else:
        print(f"Directory does not exist: {TRAIN_DIR}")

# Visualization

In [ ]:
def visualize_random_patient(patients_dict, slice_idx=None):
    """
    Visualize one folder of images with random patient.
    """
    # Pick a random patient
    patient_id = random.choice(list(patients_dict.keys()))
    data = patients_dict[patient_id]
    modalities = ['T1', 'T1CE', 'T2', 'FLAIR', 'SEG']

    # Use middle slice if none given
    if slice_idx is None:
        slice_idx = data['T1'].shape[2] // 2

    # Plot all modalities
    fig, axes = plt.subplots(1, 5, figsize=(22, 5))
    for i, mod in enumerate(modalities):
        img = data[mod][:, :, slice_idx]
        axes[i].imshow(img.T, cmap='gray' if mod != 'SEG' else 'jet', origin='lower')
        axes[i].set_title(mod)
        axes[i].axis('off')

    plt.suptitle(f"Patient: {patient_id} | Slice {slice_idx}", fontsize=16)
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.show()

visualize_random_patient(patients)

In [ ]:
# Get the first available patient
patient_key = list(patients.keys())[0]
slice_idx = 77
flair = patients[patient_key]['FLAIR']
seg = patients[patient_key]['SEG']

plt.figure(figsize=(12, 6))
plt.imshow(flair[:, :, slice_idx], cmap='gray')
plt.imshow(seg[:, :, slice_idx], cmap='jet', alpha=0.5)
plt.title(f'FLAIR + Segmentation - {patient_key}')
plt.axis('off')
plt.show()

# Apply Brain Mask

In [ ]:
masked_patients = {}

for patient_id, data in patients.items():

    #Create brain mask by combining modalities
    #Any voxel > 0 in ANY modality is considered part of the brain
    mask = np.zeros_like(data['T1'], dtype=bool)
    for mod in ['T1', 'T1CE', 'T2', 'FLAIR']:
        mask |= (data[mod] > 0)

    #Store masked data and mask image
    masked_patients[patient_id] = {
        "masked_modalities": {
            "T1":    data['T1'],
            "T1CE":  data['T1CE'],
            "T2":    data['T2'],
            "FLAIR": data['FLAIR'],
        },
        "mask_img": nib.Nifti1Image(mask.astype(np.uint8), np.eye(4)),
        "SEG" : data["SEG"]
    }

    print(f"{patient_id}: Brain voxels = {mask.sum():,}")

In [ ]:
pid = random.choice(list(masked_patients.keys()))
mask = masked_patients[pid]["mask_img"].get_fdata().astype(bool)
flair = masked_patients[pid]["masked_modalities"]["FLAIR"]

z = flair.shape[2] // 2
plt.figure(figsize=(7,7))
plt.imshow((flair * mask)[:, :, z].T, cmap='gray', origin='lower')
plt.title(f"{pid} | Brain Mask Applied")
plt.axis("off")
plt.show()

# Intensity Clipping

In [ ]:
def intensity_clipping(volume, mask=None, lower=1, upper=99):
    """
    percentile intensity clipping on an MRI volume.
    Taken from nnU-Net (Isensee et al., Nature Methods 2021).
    """

    # If no mask provided, clip across entire non-zero voxels
    if mask is None:
        mask = volume > 0

    # Extract values ONLY inside the brain (avoid background = 0)
    brain_voxels = volume[mask]

    # Compute percentile bounds
    p_low, p_high = np.percentile(brain_voxels, [lower, upper])

    # Clip intensities
    clipped = np.clip(volume, p_low, p_high)

    return clipped

# Z Score Normalization

In [ ]:
def zscore_normalization(volume, mask=None):
    """
    Z-score normalize inside the brain mask (mean=0, std=1).
    Validated by DeepCluster (ECCV 2018), FSL, SPM, FreeSurfer, and nnU-Net.
    """

    if mask is None:
        mask = volume > 0  # avoid background

    brain_voxels = volume[mask]

    mean = brain_voxels.mean()
    std  = brain_voxels.std()

    if std == 0:
        # Extremely rare, but prevents division by zero
        return volume - mean

    normed = (volume - mean) / std
    return normed

# Multi-Channel Feature Vector

In [ ]:
voxel_data = {}   # stores X matrix per patient

for pid, entry in masked_patients.items():

    mask = entry["mask_img"].get_fdata().astype(bool)
    mods = entry["masked_modalities"]

    # 1. Intensity clipping
    for mod in ["T1", "T1CE", "T2", "FLAIR"]:
        mods[mod] = intensity_clipping(mods[mod], mask=mask)

    # 2. Z-score normalization
    for mod in ["T1", "T1CE", "T2", "FLAIR"]:
        mods[mod] = zscore_normalization(mods[mod], mask=mask)

    masked_patients[pid]["masked_modalities"] = mods

# Crop Volumes

In [ ]:
def center_crop(volume, crop_size=128):
    """
    Center crop a 3D MRI volume to crop_size^3.
    """
    x, y, z = volume.shape
    cx, cy, cz = x//2, y//2, z//2
    half = crop_size // 2

    return volume[
        cx - half : cx + half,
        cy - half : cy + half,
        cz - half : cz + half
    ]

def get_bbox(mask):
    """Return bounding box of non-zero region."""
    coords = np.array(np.where(mask))
    zmin, ymin, xmin = coords.min(axis=1)
    zmax, ymax, xmax = coords.max(axis=1)
    return (zmin, zmax, ymin, ymax, xmin, xmax)

In [ ]:
cropped_patients = {}

for pid, entry in masked_patients.items():
    mods = entry["masked_modalities"]
    seg  = entry["SEG"]
    mask = entry["mask_img"].get_fdata().astype(bool)

    #Compute bounding box ONCE from brain mask
    zmin, zmax, ymin, ymax, xmin, xmax = get_bbox(mask)

    #Crop everything using SAME bounding box
    cropped_mods = {}
    for mod in ["T1", "T1CE", "T2", "FLAIR"]:
        vol = mods[mod]
        cropped_mods[mod] = vol[zmin:zmax+1,
                                ymin:ymax+1,
                                xmin:xmax+1]

    cropped_seg = seg[zmin:zmax+1,
                      ymin:ymax+1,
                      xmin:xmax+1]

    cropped_mask = mask[zmin:zmax+1,
                        ymin:ymax+1,
                        xmin:xmax+1]

    #Save them
    cropped_patients[pid] = {
        "masked_modalities": cropped_mods,
        "mask": cropped_mask,
        "SEG": cropped_seg
    }

    print(f"{pid} cropped to {cropped_mods['FLAIR'].shape}")

# ROI Selection

In [ ]:
roi_patients = {}

for pid, entry in cropped_patients.items():
    mods = entry["masked_modalities"]
    flair = mods["FLAIR"]
    t1ce  = mods["T1CE"]
    mask  = entry["mask"]     # brain mask in cropped space

    # ----------- SIMPLE ROI -----------
    # 1) Basic condition: include all non-zero FLAIR voxels inside the brain mask
    roi = (flair > 0) & mask

    # 2) Optional soft T1CE enhancement contribution
    #    Helps WT slightly, but does not over-prune like the new ROI
    t1ce_thr = np.percentile(t1ce[mask], 75)
    roi = roi | (t1ce > t1ce_thr)

    # No morphological operations
    # No connected component pruning
    # Purely intensity-based

    roi_patients[pid] = {
        "roi": roi,
        "SEG": entry["SEG"],
        "masked_modalities": mods,
        "mask": mask
    }

    print(f"{pid}: ROI voxels = {roi.sum()} / {mask.sum()}")

# Feature Extraction

In [ ]:
scaler = StandardScaler()
# pca = PCA(n_components=3, random_state=42)   # comment out to disable PCA

voxel_data = {}

for pid, entry in roi_patients.items():
    roi = entry["roi"]
    mods = entry["masked_modalities"]

    X = np.vstack([
        mods["T1"][roi],
        mods["T1CE"][roi],
        mods["T2"][roi],
        mods["FLAIR"][roi]
    ]).T

    # If you want to use pca, do pca.fit instead
    X_std = scaler.fit_transform(X)

    voxel_data[pid] = {
        "X_std": X_std,  # If using pca change to X_pca
        "roi": roi,
        "SEG": entry["SEG"],
        "masked_modalities": mods,
        "mask": entry["mask"]
    }

    print(f"{pid}: Using NON-PCA standardized features → {X_std.shape}")

# Dimensionality Reduction (Optional PCA)

In [ ]:
print("\nFitting PCA globally...")

# Fit PCA on all patients' standardized data
all_X_std = np.concatenate([entry["X_std"] for entry in voxel_data.values()])
print("All X_std shape:", all_X_std.shape)

pca = PCA(n_components=3, random_state=42)
pca.fit(all_X_std)

# Transform each patient
for pid, entry in voxel_data.items():
    X_std = entry["X_std"]
    entry["X_pca"] = pca.transform(X_std)
    print(f"{pid}: X_pca shape = {entry['X_pca'].shape}")

print("PCA transformation complete.\n")

In [ ]:
pid = list(voxel_data.keys())[0]
X_pca = voxel_data[pid]["X_pca"]

print(f"\nPCA QC for {pid}")
print("PCA component means:", X_pca.mean(axis=0))
print("PCA component stds :", X_pca.std(axis=0))

# Spectral Clustering

In [ ]:
# Spectral clustering with subsampling + label propagation

import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn.neighbors import KNeighborsClassifier
import time

# --- Hyperparameters ---
k = 4                     # number of clusters
SUBSAMPLE_SIZE = 20000    # max voxels per patient for spectral step
N_NEIGHBORS_GRAPH = 30    # neighbors for k-NN affinity graph (spectral)
N_NEIGHBORS_PROP = 5      # neighbors for k-NN label propagation

rng = np.random.default_rng(546)

for pid, entry in voxel_data.items():
    X_pca = entry["X_pca"]    # shape (N_voxels, 3)
    roi   = entry["roi"]      # boolean / index mask into 3D mask
    mask  = entry["mask"]     # 3D brain mask

    N = X_pca.shape[0]
    print(f"\n=== {pid} ===")
    print(f"Total voxels in ROI: {N}")

    if N < k:
        print(f"  [SKIP] N={N} < k={k}, not enough voxels to cluster.")
        continue

    # --- 1) Subsample voxels for spectral clustering ---
    if N > SUBSAMPLE_SIZE:
        idx_sub = rng.choice(N, size=SUBSAMPLE_SIZE, replace=False)
    else:
        idx_sub = np.arange(N)

    X_sub = X_pca[idx_sub]
    print(f"  Using subset of {X_sub.shape[0]} voxels for spectral clustering.")

    # --- 2) Spectral clustering on the subset (k-NN graph) ---
    spectral = SpectralClustering(
        n_clusters=k,
        eigen_solver="arpack",
        affinity="nearest_neighbors",
        n_neighbors=N_NEIGHBORS_GRAPH,
        assign_labels="kmeans",
        n_init=5,
        random_state=0,
    )

    t0 = time.time()
    try:
        labels_sub = spectral.fit_predict(X_sub).astype(np.int32)
    except MemoryError:
        print("  [ERROR] MemoryError in spectral clustering on subset. "
              "Try reducing SUBSAMPLE_SIZE or N_NEIGHBORS_GRAPH.")
        continue
    t1 = time.time()
    print(f"  Spectral (subset) done in {t1 - t0:.1f} s")

    # --- 3) Label propagation with k-NN classifier ---
    clf = KNeighborsClassifier(n_neighbors=N_NEIGHBORS_PROP)
    clf.fit(X_sub, labels_sub)

    t0 = time.time()
    labels_full = clf.predict(X_pca).astype(np.int32)
    t1 = time.time()
    print(f"  k-NN label propagation done in {t1 - t0:.1f} s")

    # Store 1D labels (all ROI voxels)
    entry["spectral_labels"] = labels_full

    # --- 4) Map labels back to 3D volume ---
    labels_3d = np.zeros_like(mask, dtype=np.int32)
    labels_3d[roi] = labels_full
    entry["spectral_labels_3d"] = labels_3d

    print("  Stored 'spectral_labels' and 'spectral_labels_3d'.")


In [ ]:
# Pick a patient to visualize
pid = list(voxel_data.keys())[1]  # or any specific ID
entry = voxel_data[pid]
flair_vol = entry["masked_modalities"]["FLAIR"]
labels_vol = entry["spectral_labels_3d"]

slice_idx = flair_vol.shape[2] // 2

plt.figure(figsize=(12, 6))
plt.imshow(flair_vol[:, :, slice_idx].T, cmap="gray", origin="lower")
plt.imshow(labels_vol[:, :, slice_idx].T,
           cmap="viridis",
           alpha=0.4,
           origin="lower")
plt.title(f"{pid} | Spectral clusters (k={k}) on FLAIR")
plt.axis("off")
plt.show()

# Evaluation: Cluster Alignment and Dice Scores

In [ ]:
# --- Dice helper ---
def dice(pred, true):
    pred = pred.astype(bool)
    true = true.astype(bool)

    if pred.sum() + true.sum() == 0:
        return 1.0
    if pred.sum() == 0 or true.sum() == 0:
        return 0.0

    return 2 * np.sum(pred & true) / (pred.sum() + true.sum())

In [ ]:
# 1) Align Spectral clusters → GT labels (per patient)
spectral_aligned_labels = {}

for pid, entry in voxel_data.items():
    print(f"\nAligning Spectral clusters for {pid}")

    roi = entry["roi"]
    spectral_labels = entry["spectral_labels"]        # shape (N_voxels,)

    # Get SEG in ROI space
    seg_vol = entry["SEG"]
    seg_flat = seg_vol[roi].astype(int)  # shape (N_voxels,)

    if len(seg_flat) != len(spectral_labels):
        print(f"Length mismatch for {pid}: seg={len(seg_flat)}, spectral={len(spectral_labels)}")
        continue

    # Map BraTS labels {1,2,4} → {0,1,2}, everything else → -1 (ignore)
    seg_map = {1: 0, 2: 1, 4: 2}
    true_seg = np.array([seg_map.get(int(v), -1) for v in seg_flat])

    # Only use voxels with valid GT labels for alignment
    valid_idx = true_seg >= 0
    true_valid = true_seg[valid_idx]
    pred_valid = spectral_labels[valid_idx]

    # Confusion matrix rows = GT, cols = clusters
    cm = confusion_matrix(true_valid, pred_valid, labels=[0, 1, 2])

    # Hungarian matching to find best cluster → GT mapping
    row_ind, col_ind = linear_sum_assignment(-cm)  # maximize agreement
    mapping = dict(zip(col_ind, row_ind))
    print(f"{pid}: cluster→GT mapping: {mapping}")

    # Apply mapping to ALL voxels
    aligned = np.array([mapping.get(int(c), -1) for c in spectral_labels])
    spectral_aligned_labels[pid] = aligned

In [ ]:
# 2) Compute Dice scores (WT, TC, ET) for each patient
dice_scores_spectral = {}

for pid, entry in voxel_data.items():
    if pid not in spectral_aligned_labels:
        continue

    print(f"\nComputing Dice for Spectral – {pid}")

    roi = entry["roi"]
    seg_vol = entry["SEG"]
    seg_flat = seg_vol[roi].astype(int)

    seg_map = {1: 0, 2: 1, 4: 2}
    true_seg = np.array([seg_map.get(int(v), -1) for v in seg_flat])

    pred_seg = spectral_aligned_labels[pid]

    if len(pred_seg) != len(true_seg):
        print(f"Shape mismatch: pred={len(pred_seg)}, true={len(true_seg)}")
        continue

    # --- Region definitions ---

    # Whole tumor: any non-background tumor label
    true_WT = true_seg > 0
    pred_WT = pred_seg > 0

    # Tumor core
    true_TC = np.isin(true_seg, [0, 2])
    pred_TC = np.isin(pred_seg, [0, 2])

    # Enhancing tumor: label 2
    true_ET = (true_seg == 2)
    pred_ET = (pred_seg == 2)

    WT = dice(pred_WT, true_WT)
    TC = dice(pred_TC, true_TC)
    ET = dice(pred_ET, true_ET)

    dice_scores_spectral[pid] = (WT, TC, ET)
    print(f"{pid}: WT={WT:.4f}, TC={TC:.4f}, ET={ET:.4f}")

# Results Summary

In [ ]:
def summarize_single_method(dice_scores, method_name="Method"):

    # Collect into arrays
    WT = np.array([v[0] for v in dice_scores.values()])
    TC = np.array([v[1] for v in dice_scores.values()])
    ET = np.array([v[2] for v in dice_scores.values()])

    summary = {
        "Method": method_name,
        "WT_mean": WT.mean(),   "WT_median": np.median(WT), "WT_std": WT.std(),
        "WT_min": WT.min(),     "WT_max": WT.max(),
        "TC_mean": TC.mean(),   "TC_median": np.median(TC), "TC_std": TC.std(),
        "TC_min": TC.min(),     "TC_max": TC.max(),
        "ET_mean": ET.mean(),   "ET_median": np.median(ET), "ET_std": ET.std(),
        "ET_min": ET.min(),     "ET_max": ET.max(),
    }

    df = pd.DataFrame([summary])
    display(df)
    print("\nLaTeX table:\n")
    print(df.to_latex(index=False, float_format="%.4f"))
    return df


_ = summarize_single_method(dice_scores_spectral, "Spectral")

In [ ]:
import random

def visualize_fcm_result(pid, slice_axis='z'):
    """
    Visualize ground-truth vs Spectral clustering segmentation for a given patient.
    slice_axis: 'z', 'y', or 'x'
    """

    mods  = cropped_patients[pid]["masked_modalities"]
    flair = mods["FLAIR"]
    seg   = cropped_patients[pid]["SEG"]
    roi   = voxel_data[pid]["roi"]

    pred  = spectral_aligned_labels[pid]

    # Convert ROI + predictions back into arrays
    pred_full = np.zeros_like(seg) - 1
    pred_full[roi] = pred

    # Choose slice
    if slice_axis == 'z':
        slice_idx = flair.shape[2] // 2
        flair_slice = flair[:,:,slice_idx]
        seg_slice   = seg[:,:,slice_idx]
        pred_slice  = pred_full[:,:,slice_idx]
    elif slice_axis == 'y':
        slice_idx = flair.shape[1] // 2
        flair_slice = flair[:,slice_idx,:]
        seg_slice   = seg[:,slice_idx,:]
        pred_slice  = pred_full[:,slice_idx,:]
    else:
        slice_idx = flair.shape[0] // 2
        flair_slice = flair[slice_idx,:,:]
        seg_slice   = seg[slice_idx,:,:]
        pred_slice  = pred_full[slice_idx,:,:]

    # Images
    plt.figure(figsize=(14,4))

    plt.subplot(1,3,1)
    plt.title(f"{pid} \u2014 FLAIR")
    plt.imshow(flair_slice.T, cmap='gray', origin='lower')
    plt.axis("off")

    plt.subplot(1,3,2)
    plt.title("Ground Truth (SEG)")
    plt.imshow(flair_slice.T, cmap='gray', alpha=0.6, origin='lower')
    plt.imshow(seg_slice.T, cmap='jet', alpha=0.4, origin='lower')
    plt.axis("off")

    plt.subplot(1,3,3)
    # Updated title to reflect Spectral Clustering
    plt.title("Spectral Clustering Prediction (aligned)")
    plt.imshow(flair_slice.T, cmap='gray', alpha=0.6, origin='lower')
    plt.imshow(pred_slice.T, cmap='jet', alpha=0.4, origin='lower')
    plt.axis("off")

    plt.tight_layout()
    plt.show()

#Set specific patient ID for visualization
specific_pid = 'BraTS20_Training_001'
print("Visualizing:", specific_pid)
visualize_fcm_result(specific_pid)